In [3]:
!pip install neo4j-driver
!pip install "graphistry[all]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 3.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j-driver: filename=neo4j_driver-5.25.0-py3-none-any.whl size=296842 sha256=a9fae313355cfa72cf01288302aaa38de1053e1f7c4c42d4a4fb4edd0928980f
  Stored in directory: /root/.cache/pip/wheels/60/81/f7/cd5c8e05080a94a5f7db484ce9ebfcc4e7f3cafaefadf665dd
Successfully built neo4j-driver
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchdata to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandas as pd
import neo4j # just for testing
from neo4j import GraphDatabase # for data loader
import graphistry
print('neo4j', neo4j.__version__)
print('graphistry', graphistry.__version__)

neo4j 5.25.0
graphistry 0.34.16


In [1]:
import subprocess   
import requests  
from requests.auth import HTTPBasicAuth

def run_command(command):  
    """Run a shell command and print the output."""  
    print(f"Running command: {command}")  
    result = subprocess.run(command, shell=True, text=True, capture_output=True)  
    print(result.stdout)  
    if result.returncode != 0:  
        print(f"Error: {result.stderr}")  
        raise Exception(f"Command failed: {command}")  

def install_neo4j():  
    # Update and upgrade the server  
    run_command("sudo apt-get update -y")  

    # Install Neo4j dependencies  
    run_command("sudo apt-get install wget curl nano software-properties-common dirmngr apt-transport-https gnupg gnupg2 ca-certificates lsb-release ubuntu-keyring unzip -y")  

    # Add the official Neo4j GPG key  
    run_command("curl -fsSL https://debian.neo4j.com/neotechnology.gpg.key | sudo gpg --dearmor -o /usr/share/keyrings/neo4j.gpg")  

    # Add the Neo4j repository  
    run_command('echo "deb [signed-by=/usr/share/keyrings/neo4j.gpg] https://debian.neo4j.com stable latest" | sudo tee -a /etc/apt/sources.list.d/neo4j.list')  

    # Update apt  
    run_command("sudo apt-get update")  

    # Install Neo4j  
    run_command("sudo apt-get install neo4j -y")

    run_command("sudo /etc/init.d/neo4j start")
    # Start and enable the Neo4j service  
    run_command("sudo update-rc.d neo4j defaults ")  

def configure_neo4j_remote_access():  
    # Edit the Neo4j configuration file to allow remote connections  
    run_command('sudo sed -i "s/#server.default_listen_address=0.0.0.0/server.default_listen_address=0.0.0.0/" /etc/neo4j/neo4j.conf')  

    # Restart the Neo4j service  
    run_command("sudo /etc/init.d/neo4j restart neo4j")  

def update_hosts_file(server_ip, hostname):  
    # Update /etc/hosts file  
    with open('/etc/hosts', 'a') as hosts_file:  
        hosts_file.write(f"{server_ip} {hostname}\n")  
    print(f"Added {server_ip} {hostname} to /etc/hosts")  

def change_password(new_password):  
    """Changes the Neo4j password using the HTTP API."""  
    url = "http://localhost:7474/user/neo4j/password"  
    response = requests.put(url, auth=HTTPBasicAuth('neo4j', 'neo4j'), json={"password": new_password})  

    if response.status_code == 200:  
        print("Password changed successfully.")  
    else:  
        print(f"Failed to change password: {response.status_code} - {response.text}")  
def main():  
    install_neo4j()  
    new_password = "12345"  
    print("Changing Neo4j password to '12345'...")  
    change_password(new_password)  
    configure_neo4j_remote_access()  
    
    # Example usage for updating hosts file and firewall  
    server_ip = "172.19.2.2"  # Replace with your server's IP address  
    hostname = "neo4j.dev"          # Replace with your desired hostname  
    update_hosts_file(server_ip, hostname) 

main()

Running command: sudo apt-get update -y
Get:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:5 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]
Get:6 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [28.6 kB]
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:8 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3339 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1274 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4235 kB]
Get:11 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1552 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [4027 kB]
Get:13 http://archive.ub

In [6]:
from neo4j import GraphDatabase

class HelloWorldExample:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def print_greeting(self, message):
        with self.driver.session() as session:
            greeting = session.execute_write(self._create_and_return_greeting, message)
            print(greeting)

    @staticmethod
    def _create_and_return_greeting(tx, message):
        result = tx.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
        return result.single()[0]


greeter = HelloWorldExample("bolt://172.19.2.2:7474", "neo4j", "12345")
greeter.print_greeting("hello, world")
greeter.close()

KeyboardInterrupt: 